In [1]:
import pandas as pd
import json

dashboard = pd.ExcelFile("data.xlsx")

In [2]:
data = pd.read_excel(dashboard,"Statistics")

aes = pd.read_excel(dashboard,"AEs")

referrals = pd.read_excel(dashboard, "Referrals")

partnerDistricts = pd.read_excel("partnerDistrict.xlsx")

In [3]:
data['totalMCs'] = data['mc15-19'] + data['mc20-24'] +data['mc25-29']+data['mc30-34']+data['mc35-39'] + data['mc40-44'] + data['mc45-49']+ data['mc50']

data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_mcs_referred_for_sti_services,total_mcs_referred_to_other_services,total_surgicalDisposable,total_surgicalReusable,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType
0,February,Border Church Clinic,submission,MARONDERA,0,0,0,MAHUSEKWA - 100829 - HOSPITAL,20,0,...,0,0,OK,OK,6,0,0.0,0,2025,0
1,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,18,0,...,0,0,OK,0,8,0,0.0,0,2025,Outreach
2,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,21,0,...,0,0,OK,0,11,0,0.0,0,2025,Outreach
3,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,11,0,...,0,0,OK,0,7,0,0.0,0,2025,Outreach
4,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,6,0,...,0,0,OK,0,3,0,0.0,0,2025,Outreach
5,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,7,0,...,0,0,OK,0,2,0,0.0,0,2025,Outreach
6,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,26,0,...,0,0,OK,0,21,0,0.0,0,2025,Outreach
7,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,25,0,...,0,0,OK,0,16,0,0.0,0,2025,Outreach
8,0,0,0,HURUNGWE,0,0,0,KAROI - 100715 - DISTRICT HOSPITAL,43,0,...,0,0,OK,0,14,0,0.0,0,2025,Static
9,0,0,0,HURUNGWE,0,0,0,TENGWE - 101886 - CLINIC,19,0,...,0,0,OK,0,6,0,0.0,0,2025,Static


In [4]:
aes

,AERecordingMonth,AERecordingSite,AERecordingYear,District,aeComments,ae_classification,ae_type_code,circumcising_cadre,client_age,date_ae_identified,mcMethod,vmmc_number
0,February,Border Church Clinic,submission,MARONDERA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,February,Turf Clinic,2025,MHONDORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,December,Donain,2024,MHONDORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,March,Karoi District Hospital,2025,HURUNGWE,Encouraged to elevate penis to promote venous ...,Moderate,Swelling/ moderate,Nurse,16.0,2025-03-05,Surgical,68.0


In [5]:
# Ensure Month-Year columns are strings for easy comparison
data['year'] = data['year'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(str)

data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)

# Transform df1 to include the list object
json_data = []

for _, row in data.iterrows():
    row_data = row.to_dict()
    print(type(row['totalAES']))
    if row['totalAES'] >= 1:
        # Filter df2 to find matching rows
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        print(recordedAEs)
        
        # Add list object to the row if matches are found
        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        # Add an empty list if no matches
        row_data['matchingAES'] = []


    if row['referralToOtherServices'] >= 1:
        # Filter df2 to find matching rows
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]


        # Add list object to the row if matches are found
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        # Add an empty list if no matches
        row_data['otherReferrals'] = []
    
    # Append to the final JSON structure
    json_data.append(row_data)

# Convert to JSON
result_json = json.dumps(json_data, indent=4)

output_file = "data.json"

# Save the JSON data to the file
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(result_json)

<class 'int'>
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, aeComments, ae_classification, ae_type_code, circumcising_cadre, client_age, date_ae_identified, mcMethod, vmmc_number]
Index: []
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
  AERecordingMonth AERecordingSite AERecordingYear  District aeComments  \
1         February    Turf  Clinic            2025  MHONDORO        NaN   

  ae_classification ae_type_code circumcising_cadre  client_age  \
1               NaN          NaN                NaN         NaN   

  date_ae_identified mcMethod  vmmc_number  
1                NaN      NaN          NaN  
<class 'int'>
<class 'int'>
  AERecordingMonth AERecordingSite AERecordingYear  District aeComments  \
2         December     